In [23]:
import gzip
from collections import defaultdict
import re
import numpy
import matplotlib.pyplot as plt
import sklearn.linear_model as skl
import math
import os

def readGz(f):
  for l in gzip.open(f):
    yield eval(l)

In [27]:
reviews = []

for l in readGz("/Users/tusharkoul/Desktop/Studies/CSE258/assignment1/train.json.gz"):
    reviews.append(l)

reviews_train = reviews[:100000]
reviews_validation = reviews[100000:]

'/assignment1/train.json.gz'

In [42]:
def feature(review):
    reviewWordCounts = len(re.findall(r'\w+', review['reviewText']))
    reviewVoteCount = review['helpful']['outOf']

    itemStarRating = review['rating']
    itemCategoryId = review['categoryID']

    yearMonthFeat = yearMonthOneHotEncoding(review)
    genderFeat = genderCategoryEncoding(review)
    return [1, reviewWordCounts, reviewVoteCount, reviewVoteCount**2, itemStarRating, itemCategoryId] \
           + yearMonthFeat + genderFeat


def yearMonthOneHotEncoding(review):
    # one hot encoding for years from 2003 to 2014
    reviewTime = review['reviewTime'].split()
    reviewYear = int(reviewTime[2])
    reviewMonth = int(reviewTime[0])

    yearFeat = [0] * 12
    baseYear = 2003
    yearIndex = reviewYear - baseYear
    yearFeat[yearIndex] = 1

    # encoding month, no need to encode december
    # all 0's means december
    monthFeat = [0] * 11
    if reviewMonth < 11 :
        monthFeat[reviewMonth-1] = 1
    return yearFeat + monthFeat

# Women = 1, Men = 2, Boys = 3, Girls = 4
def genderCategoryEncoding(review):
    feature = [0] * 5
    for categoryList in review['categories']:
        if 'Women' in categoryList:
            feature[0] = 1
            break
        elif 'Men' in categoryList:
            feature[1] = 1
            break
        elif 'Boys' in categoryList:
            feature[2] = 1
            break
        elif 'Girls' in categoryList:
            feature[3] = 1
            break
        elif 'Baby' in categoryList:
            feature[4] = 1
            break
    return feature

In [10]:
# TRAINING FOR LOGISTIC REGRESSION
# BROKEN DOWN BY outOf -
# 0 - 10
# 11 - 100
# 101 - 200
# 200 -
X_train1 = [feature(review) for review in reviews_train if 0 < review['helpful']['outOf'] <= 10]
Y_train1 = [round(review['helpful']['nHelpful'] / review['helpful']['outOf']) for review in reviews_train
           if 0 < review['helpful']['outOf'] <= 10]
weights1 = [review['helpful']['outOf'] for review in reviews_train
            if 0 < review['helpful']['outOf'] <= 10]

X_train2 = [feature(review) for review in reviews_train if 10 < review['helpful']['outOf'] <= 100]
Y_train2 = [round(review['helpful']['nHelpful'] / review['helpful']['outOf']) for review in reviews_train
           if 10 < review['helpful']['outOf'] <= 100]
weights2 = [review['helpful']['outOf'] for review in reviews_train
            if 10 < review['helpful']['outOf'] <= 100]

X_train3 = [feature(review) for review in reviews_train if 100 < review['helpful']['outOf'] <= 200]
Y_train3 = [round(review['helpful']['nHelpful'] / review['helpful']['outOf']) for review in reviews_train
           if 100 < review['helpful']['outOf'] <= 200]
weights3 = [review['helpful']['outOf'] for review in reviews_train
            if 100 < review['helpful']['outOf'] <= 200]

X_train4 = [feature(review) for review in reviews_train if review['helpful']['outOf'] > 200]
Y_train4 = [round(review['helpful']['nHelpful'] / review['helpful']['outOf']) for review in reviews_train
           if review['helpful']['outOf'] > 200]
weights4 = [review['helpful']['outOf'] for review in reviews_train
            if review['helpful']['outOf'] > 200]

In [48]:
# MULTIPLE MODELS

# FITTING USING LINEAR REGRESSION WITH REGULARIZATION
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
# weights = [review['helpful']['outOf'] for review in reviews_train if review['helpful']['outOf'] > 0 ]

clf = skl.LogisticRegression()
clf.fit(X_train1, Y_train1, weights1)
# clf.fit(X_train1, Y_train1)
theta1 = clf.coef_

# clf.fit(X_train2, Y_train2)
clf.fit(X_train2, Y_train2, weights2)
theta2 = clf.coef_

# clf.fit(X_train3, Y_train3)
clf.fit(X_train3, Y_train3, weights3)
theta3 = clf.coef_

# clf.fit(X_train4, Y_train4)
clf.fit(X_train4, Y_train4, weights4)
theta4 = clf.coef_

In [50]:
# Error on Validation (LOGISTIC W/O REGULARIZATION)
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

theta1_matrix = numpy.matrix(numpy.array(theta1))
theta2_matrix = numpy.matrix(numpy.array(theta2))
theta3_matrix = numpy.matrix(numpy.array(theta3))
theta4_matrix = numpy.matrix(numpy.array(theta4))

predictions_validation = []
for review in reviews_validation:
    X = feature(review)
    X = numpy.matrix(X)
    # prediction = -1
    if 0 < review['helpful']['outOf'] <= 10:
        prediction = X * theta1_matrix.transpose()
    elif 10 < review['helpful']['outOf'] <= 100:
        prediction = X * theta2_matrix.transpose()
    elif 100 < review['helpful']['outOf'] <= 200:
        prediction = X * theta3_matrix.transpose()
    elif review['helpful']['outOf'] > 200:
        prediction = X * theta4_matrix.transpose()
    predicted_helpful = sigmoid(prediction.item(0)) * review['helpful']['outOf']
    predictions_validation.append(round(predicted_helpful))

abserrors = [abs(review['helpful']['nHelpful'] - predictedHelpful)
                for (review,predictedHelpful) in zip(reviews_validation,predictions_validation)]

mae = sum(abserrors) / len(abserrors)
print('\nMean Average Error = ',mae)


Mean Average Error =  0.1819


In [51]:
reviews_test = []
for l in readGz("/Users/tusharkoul/Desktop/Studies/CSE258/assignment1/test_Helpful.json.gz"):
    reviews_test.append(l)

predictions_test = []
for review in reviews_test:
    X = feature(review)
    X = numpy.matrix(X)
    # prediction = -1
    if 0 < review['helpful']['outOf'] <= 10:
        prediction = X * theta1_matrix.transpose()
    elif 10 < review['helpful']['outOf'] <= 100:
        prediction = X * theta2_matrix.transpose()
    elif 100 < review['helpful']['outOf'] <= 200:
        prediction = X * theta3_matrix.transpose()
    elif review['helpful']['outOf'] > 200:
        prediction = X * theta4_matrix.transpose()
    predicted_helpful = sigmoid(prediction.item(0)) * review['helpful']['outOf']
    predictions_test.append(round(predicted_helpful))

In [52]:
predictionsFile = open("/Users/tusharkoul/Desktop/Studies/CSE258/assignment1/predictions_Helpful.txt", 'w')
index = 0
for l in open("/Users/tusharkoul/Desktop/Studies/CSE258/assignment1/pairs_Helpful.txt"):
    if l.startswith("userID"):
        #header
        predictionsFile.write(l)
        continue
    u,i,outOf = l.strip().split('-')
    outOf = int(outOf)
    predictionsFile.write(u + '-' + i + '-' + str(outOf) + ',' + str(float(predictions_test[index])) + '\n')
    index += 1
predictionsFile.close()